#### 不是YOLO的实现

In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

In [ ]:
#用score筛选boxes
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=.6):
    scores_all = box_confidence * box_class_probs
    scores = K.max(scores_all, axis=-1)
    classes = K.argmax(scores_all, axis=-1)
    mask = scores > threshold
    classes = tf.boolean_mask(classes, mask)
    boxes = tf.boolean_mask(boxes, mask)
    scores = tf.boolean_mask(scores, mask)
    
    return scores, boxes, classes

#### 函数
    - K.max(x, axis=None, keepdims=False):默认会去掉筛选后的维
    - K.argmax(x, axis=-1):会去掉筛选后的维
    - [tf.boolean_mask](https://tensorflow.google.cn/api_docs/python/tf/boolean_mask)
        ```python
        #形式：
        tf.boolean_mask(
                tensor, #N-D tensor
                mask, #K-D tensor
                name='boolean_mask',
                axis=None #表示从哪个维度开始，满足K + axis <= N
            )
            return #(N-K+1)-dimentional tensor
        #Example:
           # 2-D example
            tensor = [[1, 2], [3, 4], [5, 6]]
            mask = np.array([True, False, True])
            boolean_mask(tensor, mask)  # [[1, 2], [5, 6]]

          ```

#### 方法
    - Tensor 类型也可以使用
     ```python
     Tensor < n
     ```
     获取boolen类型的mask
    

In [ ]:
#IoU定义，注意这里不是Tensor类型的
def iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = (xi2 - xi1) * (yi2 - yi1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    
    iou = float(inter_area) / union_area

    return iou

In [ ]:
#基于IoU的非极大值抑制(nms)
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    """
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))

    nms_indices = tf.image.non_max_suppression(boxes=boxes, scores=scores, 
                                              max_output_size=max_boxes_tensor,
                                              iou_threshold=iou_threshold,)
    
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

#### 函数
- 一气呵成的tensor变量初始化
    ```python
        K.get_session().run(tf.variables_initializer([
                max_boxes_tensor
            ]))
    ```
- tensorflow 的nms：[tf.image.non_max_suppression](https://tensorflow.google.cn/api_docs/python/tf/image/non_max_suppression)
    ```python
        tf.image.non_max_suppression(
            boxes, # A 2-D float Tensor of shape [num_boxes, 4]
            scores, # A 1-D float Tensor of shape [num_boxes]
            max_output_size, #输出的最多boxes
            iou_threshold=0.5,
            score_threshold=float('-inf'),
            name=None
        )
        return #A 1-D integer Tensor of shape [M]，M <= max_output_size
    ```
- 在给定的张量中检索给定下标的向量
    ```python
        K.gather(reference, indices)
#     reference：张量
#     indices：整数张量，其元素为要查询的下标
    ```

#### 方法
- tensorflow的nms方法要求max_output_size为tensor，应该加入trainable=False

In [ ]:
#构成yolo末端的筛选
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    #直接接受convNet输出
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # 将xy wh位置信息转化为角点位置信息 
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # score筛选
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    
    # 转化回xy wh 位置信息
    boxes = scale_boxes(boxes, image_shape)

    # nms
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
    
    return scores, boxes, classes

In [ ]:
sess = K.get_session()
#读取模型
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)
yolo_model = load_model("model_data/yolo.h5")


In [ ]:
#查看模型参数
yolo_model.summary()

In [ ]:
#获取yolo的输出，即，将yolo输出添加到计算图中，然后连接到之前定义的图中
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
def predict(sess, image_file):
    
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (416, 416))
    #运行计算图
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={
        yolo_model.input:image_data,K.learning_phase(): 0
    })

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

In [ ]:
#调用
out_scores, out_boxes, out_classes = predict(sess, "test.jpg")